In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

In [1]:
import pandas as pd
import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

def preprocessing(text):
    extract_except = '[^A-Za-z ]+'
    text = re.sub(extract_except," ",text)
    text = (text.lower()).replace('\t\n'," ") 
    # tokenizing
    token_text = [lemmatizer.lemmatize(word, pos = 'n') for word in word_tokenize(text) if not word in stopwords.words()]

    return token_text

feedback_data = pd.read_csv('Feedback_sheet.csv', encoding = 'ISO-8859-1')
id_feedback = feedback_data[['ID', 'Feedback']]
id_touchpoint = feedback_data[['ID', 'Touchpoint']]




In [2]:
lemmatized_sentences = []
for sentence in feedback_data['Feedback']:
    lemmatized_sentence = preprocessing(sentence)
    lemmatized_sentences.append(" ".join(lemmatized_sentence))
    

    
    
        


In [3]:
feedback_words = pd.DataFrame(lemmatized_sentences, columns = ["Feedback Words"])
feedback_words


,Feedback Words
0,careline line difficult answer fixed
1,atm rhb jasin melaka
2,grade apps
3,maintaining service like gift something good u...
4,hope improve mobile app faction link branch ac...
...,...
3449,branch staff responsive phone call email
3450,customer service rhbgroup customer advocacy rh...
3451,higher interest rate time people need
3452,card support month installment zero interest p...


In [4]:
id_touchpoint

,ID,Touchpoint
0,bedba74e-1c23-4772-9ca9-6bf81bd1b27f,&lt;item&gt;Branch&lt;/item&gt;
1,e170ee8d-1287-4f22-b597-c1f2d2b3e2cc,"&lt;item&gt;ATM, Cash Deposit Machines&lt;/ite..."
2,feb9fc8d-15c3-407b-961a-e1f09677f3d0,&lt;item&gt;RHB Mobile Banking Application&lt;...
3,22990519-8eb0-4a16-9e07-c9050c50e520,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;ATM...
4,d2122a89-2eef-43e0-b8f8-44d399c04498,&lt;item&gt;RHB Internet Banking&lt;/item&gt;
...,...,...
3449,1c07bad0-288d-456d-a307-09e90d608845,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;Cal...
3450,fdacc075-cd02-40e5-a8a7-c443192562f6,&lt;item&gt;RHB Internet Banking&lt;/item&gt;
3451,c4f22c48-9d7b-4b15-940e-bfcb55497b4a,&lt;item&gt;RHB Internet Banking&lt;/item&gt;
3452,4e86c213-2ed5-40fd-9edf-c9b2a8f229f0,&lt;item&gt;Branch&lt;/item&gt;&lt;item&gt;Cal...


In [5]:
all_customer_touchpoints = []
for touchpoint in id_touchpoint['Touchpoint']:
    split_touchpoint = touchpoint.split(';')[2:]
    resulting_touchpoint = []
    for item in split_touchpoint:
        if item not in ['item&gt','/item&gt', '&lt', '', '']:
            item = item.replace('&lt','')
            resulting_touchpoint.append(item)
    all_customer_touchpoints.append(resulting_touchpoint)
        


In [16]:
pd.DataFrame(all_customer_touchpoints)
touchpoint_list = ["Social media", "Call centre", "Branch", "RHB Internet Banking", "RHB Mobile Banking Application", "Relationship managers / Personal Bankers", 'ATM, Cash Deposit Machines']
touchpoint_df = pd.DataFrame(columns = touchpoint_list)
touchpoint_df

all_touchpoint_binary_class = []


for customer_touchpoint in all_customer_touchpoints:
    touchpoint_binary_class = ["No", "No", "No", "No", "No", "No", "No"]
    for touchpoint in customer_touchpoint:
        
        if touchpoint == 'Social media':
            touchpoint_binary_class[0] = "Yes"
            continue
        elif touchpoint == 'Call centre':
            touchpoint_binary_class[1] = "Yes"
            continue
        elif touchpoint == 'Branch':
            touchpoint_binary_class[2] = "Yes"
            continue 
        elif touchpoint == 'RHB Internet Banking':
            touchpoint_binary_class[3] = "Yes"
            continue    
        elif touchpoint == 'RHB Mobile Banking Application':
            touchpoint_binary_class[4] = "Yes"
            continue
        elif touchpoint == 'Relationship managers / Personal Bankers':
            touchpoint_binary_class[5] = "Yes"
            continue
        elif touchpoint == 'ATM, Cash Deposit Machines':
            touchpoint_binary_class[6] = "Yes"
            continue
        
                
    all_touchpoint_binary_class.append(touchpoint_binary_class)
    #print(touchpoint_binary_class)
"""touchpoint_header = [touchpoint_list]  
touchpoint_header.append(all_touchpoint_binary_class)
touchpoint_header"""
Main_df = pd.DataFrame(all_touchpoint_binary_class, columns =touchpoint_list, dtype = str)
Main_df["Feedback"] = feedback_words
Main_df["Rating"] = feedback_data['Rating']
Main_df

    
            
    
                

            
            
    
    



,Social media,Call centre,Branch,RHB Internet Banking,RHB Mobile Banking Application,Relationship managers / Personal Bankers,"ATM, Cash Deposit Machines",Feedback,Rating
0,No,No,Yes,No,No,No,No,careline line difficult answer fixed,7
1,No,No,No,No,No,No,Yes,atm rhb jasin melaka,6
2,No,No,No,No,Yes,No,Yes,grade apps,6
3,No,No,Yes,No,No,No,Yes,maintaining service like gift something good u...,10
4,No,No,No,Yes,No,No,No,hope improve mobile app faction link branch ac...,5
...,...,...,...,...,...,...,...,...,...
3449,No,Yes,Yes,Yes,No,No,No,branch staff responsive phone call email,6
3450,No,No,No,Yes,No,No,No,customer service rhbgroup customer advocacy rh...,0
3451,No,No,No,Yes,No,No,No,higher interest rate time people need,7
3452,No,Yes,Yes,No,No,No,No,card support month installment zero interest p...,7


In [7]:
Main_df.to_csv("cleaned_dataset.csv")

In [8]:
touchpoint_list

['Social media',
 'Call centre',
 'Branch',
 'RHB Internet Banking',
 'RHB Mobile Banking Application',
 'Relationship managers / Personal Bankers',
 'ATM, Cash Deposit Machines']